# Uploading our stuff

    First, lets import our python toolbox and the data

In [2]:
import pandas as pd
import psycopg2

In [39]:
dfip = pd.read_excel('InPerfectOutput.xlsx')
print(dfip.head())

   Ser     Category             SubCategory         SubSubCategory  \
0    5  Agriculture  Agricultural Equipment  Aquaculture Equipment   
1    6  Agriculture  Agricultural Equipment  Aquaculture Equipment   
2    7  Agriculture  Agricultural Equipment  Aquaculture Equipment   
3    8  Agriculture  Agricultural Equipment  Aquaculture Equipment   
4    9  Agriculture  Agricultural Equipment  Aquaculture Equipment   

             SubSubSubCategory  
0             Aquaculture Trap  
1                Fishing Float  
2                  Fishing Net  
3                 Fishing Rope  
4  Other Aquaculture Equipment  


In [40]:
dfp = pd.read_excel('PerfectOutput.xlsx')
print(dfp.head())

   Ser     Category             SubCategory            SubSubCategory  \
0    3  Agriculture  Agricultural Equipment  Agricultural Greenhouses   
1    4  Agriculture  Agricultural Equipment     Aquaculture Equipment   
2    5  Agriculture  Agricultural Equipment     Aquaculture Equipment   
3    6  Agriculture  Agricultural Equipment     Aquaculture Equipment   
4    7  Agriculture  Agricultural Equipment     Aquaculture Equipment   

  SubSubSubCategory  
0               NaN  
1               NaN  
2  Aquaculture Trap  
3     Fishing Float  
4       Fishing Net  


In [42]:
dfSlim = pd.read_excel('slimTree.xlsx')
print(dfSlim.head())

                   category               subcategory
0               Agriculture    Agricultural Equipment
1    Agricultural Equipment  Agricultural Greenhouses
2  Agricultural Greenhouses                       NaN
3               Agriculture    Agricultural Equipment
4    Agricultural Equipment     Aquaculture Equipment


In [96]:
con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

In [55]:
# select * from "Categories" c;
# insert into "Categories" (category) values ('food');

cursor.execute("""select * from "Categories" c""")
cursor.execute("""insert into "Categories" (category) values ('food but with py again this time')""")


    Read from DB to test

In [56]:
cursor.execute("""select * from "Categories" c""")

print(cursor.fetchall())

[(3, 'food2'), (4, 'food'), (6, 'food9'), (8, 'food99'), (27, 'food but with py again this time')]


    Due to a few issues, lets unit test this again

In [57]:
cursor.execute("""select * from "Categories" c""")
cursor.execute("""insert into "Categories" (category) values ('unique code which will be in table')""")

cursor.execute("""select * from "Categories" c""")
print(cursor.fetchall())

[(3, 'food2'), (4, 'food'), (6, 'food9'), (8, 'food99'), (27, 'food but with py again this time'), (28, 'unique code which will be in table')]


    Looks like reading & writing works, now lets try deleting

In [58]:
cursor.execute("""select * from "Categories" c""")
category_id = "28"; category_name = 'unique code which will be in table'
cursor.execute("""DELETE FROM "Categories" WHERE category_id = """ + category_id)# + """ OR category = """ + category_name)# type: ignore #, (category_id, category_name))

cursor.execute("""select * from "Categories" c""")
print(cursor.fetchall())

[(3, 'food2'), (4, 'food'), (6, 'food9'), (8, 'food99'), (27, 'food but with py again this time')]




    Now lets make a similar version for the categroy tree:

In [63]:
# cursor.execute("""select * from "Category_Tree" ct""")
# cursor.execute("""insert into "Category_Tree" (category_id, sub_category_id) values ("catid", "subcatid")""")

# cursor.execute("""select * from "Categories" c""")
# print(cursor.fetchall())

    this would work to pass a string, but Category_Tree's category & sub category parameters both take integers, so lets get encoding

    First lets pass an old data set from pdf processing, the following cell is recycled code

In [65]:
df1 = pd.read_excel('document.xlsx')
midpoint = len(df1) // 2 # type: ignore
df2a = df1.iloc[:midpoint].reset_index(drop=True)
df2b = df1.iloc[midpoint:].reset_index(drop=True)
df2b = df2b.drop(['Category', 'SubCategory'], axis=1) # drop na collumns
df2b = df2b.iloc[1:] # remove unneccesary row with innacurate names
df2b.columns = ['SubSubCategory', 'SubSubSubCategory']
df2a = df2a.drop(['Unnamed: 3'], axis=1) # drop
df2b['Ser'] = range(1, len(df2b) + 1) # type: ignore
df3 = pd.merge(df2a, df2b, on='Ser', suffixes=('_a', '_b'))

print(df3)

       Ser     Category             SubCategory            SubSubCategory  \
0        1  Agriculture                     NaN                       NaN   
1        2          NaN  Agricultural Equipment                       NaN   
2        3          NaN                     NaN  Agricultural Greenhouses   
3        4          NaN                     NaN     Aquaculture Equipment   
4        5          NaN                     NaN                       NaN   
...    ...          ...                     ...                       ...   
4591  4592          NaN                     NaN            Truck Steering   
4592  4593          NaN                     NaN          Truck Suspension   
4593  4594          NaN                     NaN                Truck Tire   
4594  4595          NaN                     NaN        Truck Transmission   
4595  4596          NaN                     NaN               Truck Wheel   

     SubSubSubCategory  
0                  NaN  
1                  NaN  


    Now lets try making a map from this

In [75]:
df = pd.DataFrame(df3)

def combine_non_na(row):
    return ' '.join([str(val) for val in row if val != 'na' and not isinstance(val, (int, float))])

df['combined'] = df.apply(combine_non_na, axis=1)

df = df[['Ser', 'combined']]

print(df)

       Ser                  combined
0        1               Agriculture
1        2    Agricultural Equipment
2        3  Agricultural Greenhouses
3        4     Aquaculture Equipment
4        5          Aquaculture Trap
...    ...                       ...
4591  4592            Truck Steering
4592  4593          Truck Suspension
4593  4594                Truck Tire
4594  4595        Truck Transmission
4595  4596               Truck Wheel

[4596 rows x 2 columns]


    Lets save this for safe keeping

In [76]:
df.to_excel('map.xlsx', index=False)

    Now its DB time, we need to put this map into 1 of our tables, & use the map for our other table

    Lets start out by uploading the map we just made

    First lets make sure the db is empty

In [97]:
cursor.execute("""select * from "Categories" c""")
print(cursor.fetchall())

cursor.execute("""select * from "Categories" c""")
cursor.execute("""insert into "Categories" (category) values ('food but with py again this time')""")

cursor.execute("""select * from "Categories" c""")
cursor.execute("""drop table "Categories" """)

[(3, 'food2'), (4, 'food'), (6, 'food9'), (8, 'food99'), (22, 'food999')]


In [98]:

# cursor.execute("""select * from "Categories" c""")
# cursor.execute("""insert into "Categories" (category) values ('food but with py again this time')""")

# cursor.execute("""select * from "Categories" c""")
# print(cursor.fetchall())

    Now lets touch the DB

In [99]:
def get_category_subcategory(df, index):
    category = df.loc[index, 'category']
    subcategory = df.loc[index, 'subcategory']
    return category, subcategory

for i in range(len(df)): # type: ignore
    num, name = get_category_subcategory(df, i); num = str(num)
    
    cursor.execute("""select * from "Categories" c""")
    cursor.execute("""insert into "Categories" (category_id, category) values (""" + num + """, '""" + name + """')""") # type: ignore

KeyError: 'category'


    & the final step is to use this to commit our data to the DB

In [42]:
len = dfip.size

for i in dfp:
    print("f")



f
f
f
f
f


In [ ]:
# Ser = []; Cat = []; SCat = []; SSCat = []; SSSCat = []

# for i in dfp:
#     Ser.append(dfp['Ser'])
#     Cat.append(dfp['Category'])
#     SCat.append(dfp['SubCategory'])
#     SSCat.append(dfp['SubSubCategory'])
#     SSSCat.append(dfp['SubSubSubCategory'])

# Ser; # Cat; # SCat; # SSCat; # SSSCat
